### Prototype Code : Table Generator

In [ ]:
# Table File Generator

def dataframe_init(in_data, out_data, text_data):
    # initialize data of lists.
    data = {"output" : out_data, 
           "input" : in_data,
           "instruction" : text_data}

    # Create DataFrame
    df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in data.items()]))
    # df = pd.DataFrame(data)
    return df

In [ ]:
# CSV Generator

def csv_from_test_case_batches(filename, input_column, output_column, text_column):
    cols = dataframe_init(input_column, output_column, text_column)
    cols.to_csv(f"{filename}.csv", sep='\t', encoding='utf-8', index=False, header=True)

In [ ]:
# Main Program

# FILENAME
filename = "sample_name"
# URL
url = get_url()
# DATA
data = data_scrape(url) # Returns Link Data Only For Now

# Prepare data to be plcaed in the Input Column
input_data = []
for item_set in data:
    for item in item_set:
        input_data.append(item)

# CHAIN
chain = load_model_chain()
# GENERATE TEST CASES
test_cases = create_test_cases(data, chain)

# Prepare data to be plcaed in the Output Column [Split]
split_sets = []
for test_case_set in test_cases:
    for test_case in test_case_set:
        for test in test_case:
            split_sets.append(test.split('~'))
# Prepare data to be plcaed in the Output Column [Actual Column]
output_data = []
for sp in split_sets:
    output_data.append(sp)
    
# Fine-Tuning Template
text_data = []
i = 0
while (i < len(input_data)):
    template = f"####Question:\n Generate test case for the following UI element: {input_data[i]}"
    text_data.append(template)
    i+=1
    
# CSV OUTPUT
csv_from_test_case_batches(filename, input_data, output_data, text_data)

### Version 1 : Dataset Generator Polish and System Proper Split

In [ ]:
# Pandas
import pandas as pd

# Table Dataframe Initialization
def dataframe_init(data):
    df = pd.DataFrame(dict([(key, pd.Series(value)) for key, value in data.items()]))
    return df

# CSV Generator
def csv_from_test_case_batches(filename, data):
    cols = dataframe_init(data)
    cols.to_csv(f"{filename}.csv", sep='\t', encoding='utf-8', index=False, header=True)


def create_fine_tuning_dataset(scraped_data, llm_output):
    # Dataset Generator : Resulting Dataset Columns / Parameters
    out_data = llm_output # LLM Output
    in_data = scraped_data # Scraped UI Element
    instruction_data = []
    i = 0
    while (i < len(scraped_data)):
        template = f"####Question:\n Generate test case for the following UI element: {in_data[i]}"
        instruction_data.append(template)
        i+=1

    data = {"output" : out_data, 
            "input" : in_data, 
            "instruction" : instruction_data}
    csv_from_test_case_batches("fine_tuning_dataset", data)
    return dataframe_init(data)    

##### Version 1: System Proper Table Generation

In [ ]:
# System Proper Parameters
def create_table_dataset(llm_output):
    # TODO : After Dataset Generator Development

    id = []
    objective = []
    precondition = []
    test_steps = []
    expected_result = []
    actual_result = []

    i = 0
    for test_case in llm_output:
        split_test_case = test_case.split('~')

        # Test Case ID
        id.append(i+1)

        # Test Case Objective
        objective.append(split_test_case[0])

        # Test Case Precondition
        precondition.append(split_test_case[1])

        # Test Case Steps 
        test_steps.append(split_test_case[2])

        # Test Case Expected Output
        expected_result.append(split_test_case[3])

        # Test Case Actual Result
        actual_result.append("Pass/Fail")

        i+=1

    data = {"Test Case ID" : id,
            "Objective" : objective,
            "Precondition" : precondition,
            "Test Steps" : test_steps,
            "Expected Result" : expected_result,
            "Actual Result" : actual_result}

    return dataframe_init(data)